In [1]:
import pandas
from pandas import DataFrame
%matplotlib inline
import seaborn
from sklearn.ensemble import RandomForestRegressor

from sklearn import cross_validation

import numpy
import math

In [2]:
train_df = DataFrame.from_csv("train.csv", index_col=['Store', 'Date'], parse_dates=['Date'])
train_df.head()

/opt/conda/envs/python2/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Store,Date,,,,,,,
1,2015-07-31,5,5263,555,1,1,0,1
2,2015-07-31,5,6064,625,1,1,0,1
3,2015-07-31,5,8314,821,1,1,0,1
4,2015-07-31,5,13995,1498,1,1,0,1
5,2015-07-31,5,4822,559,1,1,0,1


In [4]:
# Featurize
train_df.reset_index(inplace=True)
train_df['Date'] = train_df['Date'].map(lambda x: x.value)
def numericStateHoliday(x):
    if x == 'a':
        return 1
    elif x == 'b':
        return 2
    elif x == 'c':
        return 3
    else:
        return x
train_df['StateHoliday'] = train_df['StateHoliday'].map(lambda x: numericStateHoliday(x))

In [6]:
def train_and_score(training_df, features):

    regressor = RandomForestRegressor(n_estimators=25, n_jobs=-1)
    predict = cross_validation.cross_val_predict(regressor, training_df[features], training_df['Sales'], n_jobs=-1)
    
    score_df = training_df[['Sales']].copy()
    score_df['Predict'] = predict
    score_df = score_df[score_df['Sales'] != 0]
    score_df['PercentError'] =  (score_df['Sales'] - score_df['Predict']) / score_df['Sales']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

In [7]:
%%time

features = [col for col in train_df.columns if col != 'Sales' and col != 'Customers']
print("Score: {}".format(train_and_score(train_df, features)))

Score: 0.405152883025
CPU times: user 884 ms, sys: 632 ms, total: 1.52 s
Wall time: 57.6 s


In [12]:
customer_regressor = RandomForestRegressor(n_estimators=25, n_jobs=-1)
customer_predict = cross_validation.cross_val_predict(customer_regressor, train_df[features], train_df['Customers'], n_jobs=-1)

In [15]:
train_df['Customer-Predicted'] = customer_predict

In [17]:

features = [col for col in train_df.columns if col != 'Sales' and col != 'Customers']
print("Score: {}".format(train_and_score(train_df, features)))

Score: 0.389042864952
